In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/credential_access/host/empire_mimikatz_logonpasswords.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [2]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
(
df[['@timestamp','Hostname','SubjectUserName','ProcessName','ObjectName','AccessMask','EventID']]

[(df['Channel'].str.lower() == 'security')
    & ((df['EventID'] == 4663) | (df['EventID'] == 4656))
    & (df['ObjectName'].str.contains('.*lsass.exe', regex=True))
    & (~df['SubjectUserName'].str.endswith('.*$', na=False))
]
.head()
)

,@timestamp,Hostname,SubjectUserName,ProcessName,ObjectName,AccessMask,EventID
2459,2020-08-07T14:32:57.592Z,WORKSTATION5.theshire.local,pgustavo,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,\Device\HarddiskVolume2\Windows\System32\lsass...,0x1010,4656
2460,2020-08-07T14:32:57.592Z,WORKSTATION5.theshire.local,pgustavo,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,\Device\HarddiskVolume2\Windows\System32\lsass...,0x10,4663


In [4]:
(
df[['@timestamp','Hostname','SourceImage','TargetImage','GrantedAccess','SourceProcessGUID','CallTrace']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 10)
    & (df['TargetImage'].str.contains('.*lsass.exe', regex=True))
    & (df['CallTrace'].str.contains('.*UNKNOWN*', regex=True))
]
.head()
)

,@timestamp,Hostname,SourceImage,TargetImage,GrantedAccess,SourceProcessGUID,CallTrace
2450,2020-08-07T14:32:57.589Z,WORKSTATION5.theshire.local,C:\windows\System32\WindowsPowerShell\v1.0\pow...,C:\windows\system32\lsass.exe,0x1010,{297bc33e-65d0-5f2d-8207-000000000400},C:\windows\SYSTEM32\ntdll.dll+9c534|C:\windows...


In [7]:
(
df[['@timestamp','ProcessGuid','Image','ImageLoaded']]
    
[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 7)
    & (
        (df['ImageLoaded'].str.contains('.*samlib.dll', regex=True))
        | (df['ImageLoaded'].str.contains('.*vaultcli.dll', regex=True))
        | (df['ImageLoaded'].str.contains('.*hid.dll', regex=True))
        | (df['ImageLoaded'].str.contains('.*winscard.dll', regex=True))
        | (df['ImageLoaded'].str.contains('.*cryptdll.dll', regex=True))
    )
    & (df['@timestamp'].between('2020-06-00 00:00:00.000','2020-08-20 00:00:00.000'))
]
.groupby(['ProcessGuid','Image'])['ImageLoaded'].count().sort_values(ascending=False)
).to_frame()
# )

,,ImageLoaded
ProcessGuid,Image,
{297bc33e-65d0-5f2d-8207-000000000400},C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,4


In [15]:
imageLoadDf = (
df[['@timestamp','ProcessGuid','Image','ImageLoaded']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 7)
    & (
        (df['ImageLoaded'].str.contains('.*samlib.dll', regex=True))
        | (df['ImageLoaded'].str.contains('.*vaultcli.dll', regex=True))
        | (df['ImageLoaded'].str.contains('.*hid.dll', regex=True))
        | (df['ImageLoaded'].str.contains('.*winscard.dll', regex=True))
        | (df['ImageLoaded'].str.contains('.*cryptdll.dll', regex=True))
    )
    & (df['@timestamp'].between('2020-06-00 00:00:00.000','2020-08-20 00:00:00.000'))
]
.groupby(['ProcessGuid','Image'])['ImageLoaded'].count().sort_values(ascending=False)
).to_frame()


# processAccessDf = (
df[['@timestamp', 'Hostname', 'SourceImage', 'TargetImage', 'GrantedAccess', 'SourceProcessGUID']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 10)
    & (df['TargetImage'].str.contains('.*lsass.exe', regex=True))
    & (df['CallTrace'].str.contains('.*UNKNOWN*', regex=True))
]


(
pd.merge(imageLoadDf, processAccessDf,
    left_on = 'ProcessGuid', right_on = 'SourceProcessGUID', how = 'inner')
)

,ImageLoaded,@timestamp,Hostname,SourceImage,TargetImage,GrantedAccess,SourceProcessGUID
0,4,2020-08-07T14:32:57.589Z,WORKSTATION5.theshire.local,C:\windows\System32\WindowsPowerShell\v1.0\pow...,C:\windows\system32\lsass.exe,0x1010,{297bc33e-65d0-5f2d-8207-000000000400}


## Hunter Notes
- Looking for processes accessing LSASS with the 0x10(VmRead) rights from a non-system account is very suspicious and not as common as you might think.
- GrantedAccess code 0x1010 is the new permission Mimikatz v.20170327 uses for command "sekurlsa::logonpasswords". You can specifically look for that from processes like PowerShell to create a basic signature.
    - 0x00000010 = VMRead
    - 0x00001000 = QueryLimitedInfo
- GrantedAccess code 0x1010 is less common than 0x1410 in large environment.
- Out of all the Modules that Mimikatz needs to function, there are 5 modules that when loaded all together by the same process is very suspicious.
    - samlib.dll, vaultcli.dll, hid.dll, winscard.dll, cryptdll.dll
- For signatures purposes, look for processes accessing Lsass.exe with a potential CallTrace Pattern> /C:\Windows\SYSTEM32\ntdll.dll+[a-zA-Z0-9]{1,}|C:\Windows\System32\KERNELBASE.dll+[a-zA-Z0-9]{1,}|UNKNOWN.*/
- You could use a stack counting technique to stack all the values of the permissions invoked by processes accessing Lsass.exe. You will have to do some filtering to reduce the number of false positives. You could then group the results with other events such as modules being loaded (EID 7). A time window of 1-2 seconds could help to get to a reasonable number of events for analysis.